# Modelo 1 - FIFO

### Leer los datos de excel

In [6]:
import pandas as pd

#Funcion para leer los datos de los pacientes
def leer_datos():
    # Leer el archivo de excel
    df = pd.read_excel('DT.xlsx', sheet_name='Pacientes')
    df1 = pd.read_excel("DT.xlsx", sheet_name="Medicos")
    
    # Crear un diccionario para almacenar los datos de los pacientes
    pacientesDict = {}
    medicos = {}

    # Iterar sobre las filas del DataFrame
    for index, row in df.iterrows():
        llave = row["ID Servicio"]
        valor1 = row["Llegada"]
        valor2 = row["Atencion"]

        # Agregar valores a la lista correspondiente en el diccionario
        if llave in pacientesDict:
            pacientesDict[llave].append([valor1, valor2])
        else:
            pacientesDict[llave] = [int(valor1), int(valor2)]
    pacientes = pacientesDict

    # Iterar sobre las filas del DataFrame
    for index, row in df1.iterrows():
        llave = row["ID Medico"]
        valor1 = 0
        valor2 = 0
    
        # Agregar valores a la lista correspondiente en el diccionario
        if llave in medicos:
            medicos[llave].append([valor1, valor2])
        else:
            medicos[llave] = [int(valor1), int(valor2)]
    doctores = medicos
    
    
    return doctores, pacientes

In [7]:
doctores = leer_datos()[0]

### Asignación de los médicos

In [8]:
global doctores

import pandas as pd

# Función para asignar doctor a paciente
def asignar_doctor(atencion, minuto, estadoMedicos):

    #Actualizar el estado de los doctores
    if minuto > 0:
        
        for j in doctores:
            
            if minuto >= doctores[j][1]:
                
                doctores[j][0] = 0

    for j in doctores:
        
        # Revisar si el doctor esta disponible en orden de lista
        if doctores[j][0] == 0:
            
            # Asignar doctor al paciente
            doctores[j][0] = 1
            
            # Asignar el ultimo minuto en el que esta ocupado el doctor
            doctores[j][1] = minuto + atencion
            
            break
    else:
        
        # Si no se asignó a ningún doctor en el minuto evaluado, se busca el médico disponible más cercano
        doc_list = sorted(doctores, key=lambda k: doctores[k][1])
        
        for j in doc_list:
            
            # Revisar si el doctor esta disponible en orden de lista
            if doctores[j][0] == 0:
                
                # Asignar doctor al paciente
                doctores[j][0] = 1
                
                # Asignar el ultimo minuto en el que esta ocupado el doctor
                doctores[j][1] = minuto + atencion
                
                break

    # Actualizar el estado de los doctores en el DataFrame
    for j in doctores:
        
        if minuto == doctores[j][1]:
            
            doctores[j][0] = 0


    # Actualizar el estado de los médicos en el DataFrame
    estado = {'Minuto': minuto}

    for j in doctores:
        
        estado[f'Doctor {j}'] = 'Ocupado' if doctores[j][0] == 1 else 'Disponible'

    # Agregar el estado de los médicos al DataFrame
    estadoMedicos = estadoMedicos = pd.concat([estadoMedicos, pd.DataFrame(estado, index=[0])], ignore_index=True)

    return estadoMedicos


### Simular Pacientes

In [9]:
import pandas as pd
import openpyxl

global doctores

# Función principal para asignar médicos a pacientes
def asignacion_doctor1():
    pacientes = leer_datos()[1]

    # Crear DataFrame para el estado de los médicos
    estadoMedicos = pd.DataFrame(columns=['Minuto'] + [f'Doctor {i}' for i in doctores])

    for i in pacientes:
        # Asignar doctor a paciente
        atencion = pacientes[i][1]
        minuto = pacientes[i][0]
        estadoMedicos = asignar_doctor(atencion, minuto, estadoMedicos)
    
    # Guardar el dataframe en la hoja GANTT del archivo Excel
    book = openpyxl.load_workbook('DT.xlsx')
    writer = pd.ExcelWriter('DT.xlsx', engine='openpyxl')
    writer.book = book
    estadoMedicos.to_excel(writer, sheet_name='GANTT', index=False)
    writer.save()

    return estadoMedicos

# Prueba para 02/06/2022

In [10]:
estadoMedicos = asignacion_doctor1()

/var/folders/m6/xx2tl5h91wbf651lwjmb3f900000gn/T/ipykernel_6678/1649477395.py:22: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/var/folders/m6/xx2tl5h91wbf651lwjmb3f900000gn/T/ipykernel_6678/1649477395.py:24: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
